In [18]:
from lumibot.brokers import Alpaca
from lumibot.entities import Asset
from lumibot.backtesting import YahooDataBacktesting 
from lumibot.backtesting.alpaca_backtesting import AlpacaBacktesting , DataSourceBacktesting, AlpacaData
from lumibot.backtesting.alpha_vantage_backtesting import AlphaVantageBacktesting
from lumibot.backtesting.pandas_backtesting import PandasDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime 
from alpaca_trade_api import REST 
from timedelta import Timedelta 
from finbert_utils import estimate_sentiment

from dotenv import load_dotenv
import os

load_dotenv()

True

In [ ]:
from lumibot.data_sources import AlpacaData, DataSourceBacktesting


class AlpacaBacktesting2(DataSourceBacktesting, AlpacaData):
    def __init__(self, datetime_start, datetime_end, **kwargs):
        # raise Exception("AlpacaBacktesting is not currently operational")

        AlpacaData.__init__(self, **kwargs)
        DataSourceBacktesting.__init__(self, datetime_start, datetime_end)

In [2]:
API_KEY = os.getenv("ALPACA_API_KEY") 
API_SECRET = os.getenv("ALPACA_SECRET_KEY") 
BASE_URL = os.getenv("ALPACA_API_URL")


ALPACA_CREDS = {
    "API_KEY":API_KEY, 
    "API_SECRET": API_SECRET, 
    "PAPER": True
}

In [3]:
class MLTrader(Strategy): 
    def initialize(self, symbol:str="SPY", cash_at_risk:float=.5): 
        self.symbol = symbol
        self.sleeptime = "24H" 
        self.last_trade = None 
        self.cash_at_risk = cash_at_risk
        self.api = REST(base_url=BASE_URL, key_id=API_KEY, secret_key=API_SECRET)

    def position_sizing(self): 
        cash = self.get_cash() 
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price,0)
        return cash, last_price, quantity

    def get_dates(self): 
        today = self.get_datetime()
        three_days_prior = today - Timedelta(days=3)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')

    def get_sentiment(self): 
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol, 
                                 start=three_days_prior, 
                                 end=today) 
        news = [ev.__dict__["_raw"]["headline"] for ev in news]
        probability, sentiment = estimate_sentiment(news)
        return probability, sentiment 

    def on_trading_iteration(self):
        cash, last_price, quantity = self.position_sizing() 
        probability, sentiment = self.get_sentiment()

        if cash > last_price: 
            if sentiment == "positive" and probability > .999: 
                if self.last_trade == "sell": 
                    self.sell_all() 
                order = self.create_order(
                    self.symbol, 
                    quantity, 
                    "buy", 
                    type="bracket", 
                    take_profit_price=last_price*1.20, 
                    stop_loss_price=last_price*.95
                )
                self.submit_order(order) 
                self.last_trade = "buy"
            elif sentiment == "negative" and probability > .999: 
                if self.last_trade == "buy": 
                    self.sell_all() 
                order = self.create_order(
                    self.symbol, 
                    quantity, 
                    "sell", 
                    type="bracket", 
                    take_profit_price=last_price*.8, 
                    stop_loss_price=last_price*1.05
                )
                self.submit_order(order) 
                self.last_trade = "sell"

In [17]:
start_date = datetime(2023,12,15)
end_date = datetime(2023,12,31) 
broker = Alpaca(ALPACA_CREDS) 
symbol_yahoo = "ETH-USD"
symbol_alpaca = "ETH/USD"
benchmark_asset = Asset(symbol=symbol_yahoo, asset_type="stock")
strategy = MLTrader(name='mlstrat', broker=broker,
                    parameters={"symbol":"ETH/USD", 
                                "cash_at_risk":.5})
strategy.backtest(
    YahooDataBacktesting, 
    start_date, 
    end_date, 
    benchmark_asset=benchmark_asset,
    parameters={"symbol":"ETH-USD", "cash_at_risk":.5}
)

Starting backtest for MLTrader...
Progress || 100.00%  [Elapsed: 0:37:55 ETA: 0:00:00] Portfolio Val: 100,000.000
Creating trades plot...

Creating indicators plot...

Creating tearsheet...
2024-01-24 19:13:48,238: root: ERROR: Not enough data to create a tearsheet, at least 2 days of data are required. Skipping


{'cagr': 0.0,
 'volatility': 0.0,
 'sharpe': 0,
 'max_drawdown': {'drawdown': 0.0,
  'date': Timestamp('2023-12-16 09:30:00-0500', tz='America/New_York')},
 'romad': 0,
 'total_return': 0.0}

In [15]:
import yfinance as yf

# Ethereum의 데이터를 불러옵니다
eth_data = yf.Ticker("ETH-USD")

# 지정된 기간 동안의 역사적 데이터를 가져옵니다. 예를 들어, 최근 1년간의 일일 데이터
eth_history = eth_data.history(period="1y")

# 데이터 출력
print(eth_history)

                                  Open         High          Low        Close  \
Date                                                                            
2023-01-24 00:00:00+00:00  1627.848267  1639.723877  1551.389771  1556.604248   
2023-01-25 00:00:00+00:00  1556.807495  1632.241699  1530.797852  1611.711060   
2023-01-26 00:00:00+00:00  1611.080933  1626.198242  1586.598145  1603.105957   
2023-01-27 00:00:00+00:00  1603.080078  1617.000854  1565.244995  1598.156494   
2023-01-28 00:00:00+00:00  1598.125366  1604.704102  1565.390137  1572.435059   
...                                ...          ...          ...          ...   
2024-01-20 00:00:00+00:00  2489.847656  2489.847656  2456.095703  2469.589111   
2024-01-21 00:00:00+00:00  2469.798584  2479.760498  2452.377686  2453.913086   
2024-01-22 00:00:00+00:00  2454.987305  2463.447266  2303.502686  2310.826416   
2024-01-23 00:00:00+00:00  2310.951660  2348.031250  2167.282471  2240.686035   
2024-01-24 00:00:00+00:00  2